## Figure 4 reproduction

In this notebook, we are reproducing figure 4, GRU-Attn-IEC deep learning approach we used in paper.  
It will be just "forward pass" of this model so you can play around and print shapes and see how it goes under the hood.

<img src="paper_figures/gru-attn-iec.png" alt="Figure 4" width="400"/>

In [3]:
import torch
import torch
import torch.nn as nn
import numpy

In [8]:
ie_input_size = 9 # euv visual features and in situ input
c_input_size = 4 # coronagraph visual features
hidden_size = 64
hidden_size2 = 64
output_size = 1
num_gru_layers = 2
num_heads = 4

# random tensors -> inputs
x = torch.randn(32, 100, 9)
x_q = torch.randn(32, 100, 4)

# torch layers
gru1 = nn.GRU(ie_input_size, hidden_size, num_gru_layers, batch_first=True)

gru2 = nn.GRU(16, hidden_size2, num_gru_layers, batch_first=True)
linear1 = nn.Linear(c_input_size, 16)
gelu = nn.GELU()
mha = nn.MultiheadAttention(embed_dim=hidden_size2, num_heads=num_heads, batch_first=True)

fc_out = nn.Linear(hidden_size + hidden_size2, output_size)

# forward pass
h0 = torch.zeros(num_gru_layers, x.size(0), hidden_size)
out1, _ = gru1(x, h0)

x_q = gelu(linear1(x_q))
h02 = torch.zeros(num_gru_layers, x_q.size(0), hidden_size2)
out2, _ = gru2(x_q, h02)

context, _ = mha(out2[:, -1:, :], out2, out2)
context = context.squeeze(1)

concated = torch.cat((out1[:, -1, :], context), 1)
out = fc_out(concated)

print(f"output: {out.shape}")

output: torch.Size([32, 1])
